# 自動機械学習 Automated Machine Learning による離反予測モデリング & モデル解釈 (リモート高速実行)


顧客属性データを用いて与信管理モデルを開発します。

1. 事前準備
    - Python SDK のインポート
    - Azure ML `Workspace` への接続
    - `Experiment` の作成
    - `Dataset` の作成と登録
2. 自動機械学習 Automated Machine Learning
    - 計算環境 `Machine Learning Compute` の準備
    - 自動機械学習 Automated ML の事前設定
    - モデル学習と結果の確認
3. モデル解釈

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning の Python SDKをインポートします

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

Azure Machine Learning Python SDK のバージョンを確認します。

In [2]:
print(azureml.core.VERSION)

1.0.76


In [3]:
# その他必要なライブラリを利用します
import logging
from matplotlib import pyplot as plt
import pandas as pd
import os
import warnings

# 警告メッセージを削除する
warnings.showwarning = lambda *args, **kwargs: None

### Azure ML workspace との接続
Azure Machine Learning との接続を行います。Azure Active Directory の認証が必要です。

In [4]:
# 明示的に接続情報を記載する場合
subscription_id = '9c0f91b8-eb2f-484c-979c-15848c098a6b'
resource_group = 'iothack'
workspace_name = 'azureml-iot'
workspace = Workspace(subscription_id, resource_group, workspace_name)

In [ ]:
# 保存されている接続情報を利用する場合
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

### 実験名の設定
Azure Machine Learing では 実験を管理する仕組みがあります。自動機械学習は自動的にその実験管理の仕組みでメトリックやログが残ります。

In [6]:
# choose a name for experiment
experiment_name = 'automl-classification-hmeq-dev'
experiment=Experiment(workspace, experiment_name)

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning service の Dataset のフォーマットでデータを定義します。

In [9]:
dataset = Dataset.get_by_name(workspace, name='hmeq')
dataset.take(5).to_pandas_dataframe()

,不履行フラグ,融資依頼金額,未払担保金額,現在資産価値,債務理由,職種,勤務年数,信用調査会社問い合わせ数,延滞トレードライン数,最も古いトレードラインの月齢,最近のクレジットの問い合わせ数,トレード(クレジット)ラインの数,債務対所得の割合
0,1,1100,25860.00,39025.00,HomeImp,Other,10.50,0.00,0.00,94.37,1.00,9.00,None
1,1,1300,70053.00,68400.00,HomeImp,Other,7.00,0.00,2.00,121.83,0.00,14.00,None
2,1,1500,13500.00,16700.00,HomeImp,Other,4.00,0.00,0.00,149.47,1.00,10.00,None
3,1,1500,nan,nan,,,nan,nan,nan,nan,nan,nan,None
4,0,1700,97800.00,112000.00,HomeImp,Office,3.00,0.00,0.00,93.33,0.00,14.00,None


In [10]:
#予測変数の指定
label = '不履行フラグ'

### 学習データとテストデータに分割
※ハンズオンの都合上85%のデータをモデル学習で利用します。テストデータは後半のモデル解釈で利用されます。

In [38]:
train_dataset, test_dataset = dataset.random_split(0.95, seed=12345)

In [39]:
train_dataset.to_pandas_dataframe().tail()

,不履行フラグ,融資依頼金額,未払担保金額,現在資産価値,債務理由,職種,勤務年数,信用調査会社問い合わせ数,延滞トレードライン数,最も古いトレードラインの月齢,最近のクレジットの問い合わせ数,トレード(クレジット)ラインの数,債務対所得の割合
5656,0,88900,48919.00,93371.00,DebtCon,Other,15.00,0.00,1.00,205.65,0.00,15.00,34.82
5657,0,88900,57264.00,90185.00,DebtCon,Other,16.00,0.00,0.00,221.81,0.00,16.00,36.11
5658,0,89000,54576.00,92937.00,DebtCon,Other,16.00,0.00,0.00,208.69,0.00,15.00,35.86
5659,0,89200,54045.00,92924.00,DebtCon,Other,15.00,0.00,0.00,212.28,0.00,15.00,35.56
5660,0,89900,48811.00,88934.00,DebtCon,Other,15.00,0.00,0.00,219.60,0.00,16.00,34.57


In [40]:
train_dataset.to_pandas_dataframe().describe()

,不履行フラグ,融資依頼金額,未払担保金額,現在資産価値,勤務年数,信用調査会社問い合わせ数,延滞トレードライン数,最も古いトレードラインの月齢,最近のクレジットの問い合わせ数,トレード(クレジット)ラインの数,債務対所得の割合
count,5661.00,5661.00,5153.00,5558.00,5175.00,4986.00,5113.00,5365.00,5180.00,5448.00,4450.00
mean,0.20,18623.12,73806.08,101810.50,8.93,0.25,0.45,179.75,1.19,21.26,33.76
std,0.40,11198.94,44328.98,57551.42,7.58,0.85,1.13,85.51,1.72,10.10,8.67
min,0.00,1100.00,2063.00,8000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52
25%,0.00,11100.00,46423.00,66000.00,3.00,0.00,0.00,115.19,0.00,14.00,29.12
50%,0.00,16400.00,65077.00,89208.00,7.00,0.00,0.00,173.32,1.00,20.00,34.77
75%,0.00,23300.00,91731.00,119961.50,13.00,0.00,0.00,231.73,2.00,26.00,38.97
max,1.00,89900.00,399550.00,855909.00,41.00,10.00,15.00,1168.23,17.00,71.00,203.31


### Dataset として登録
Azure Machine Learning の データセット管理機能 Dataset に登録します。

In [41]:
train_dataset = train_dataset.register(workspace, name = 'hmeq-train', description = 'hmeq train data in bootcamp', create_new_version=True)
test_dataset = test_dataset.register(workspace, name = 'hmeq-test', description = 'hmeq test data in bootcamp', create_new_version=True)

### 計算環境 (Machine Learning Compute) の設定

Azure Machine Learning は高速にモデル学習を高速に回す計算環境 Machine Learning Compute が利用できます。

In [42]:
# 予め Azure ML studio にて cpucluster という名称の Machine Learning Compute を 作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(workspace, "cpucluster")

## 2. 自動機械学習 Automated Machine Learning
自動機械学習の事前設定を行います。[AutoMLConfig](https://docs.microsoft.com/ja-JP/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) を用いて定義します。

In [43]:
automl_settings = {
    "iteration_timeout_minutes": 5, # 各試行の最大実験時間
    "iterations": 10,  # 試行する機械学習パイプラインの数
    #"max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 3,
    "primary_metric": 'accuracy', # 精度指標
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             training_data = train_dataset,
                             label_column_name = label,                             
                             #compute_target=compute_target,
                             model_explainability = True,
                             **automl_settings
                            )

### 実行と結果確認

※ Machine Learning Compute を利用する場合、一番最初の実行は Docker Image を作成するため、20 〜 30分ほど時間がかかります。Dokcer Image を Build している様子は統合開発画面 Azure Machine Learning studio から確認できます。

In [44]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_bbacf504-e49c-47d3-9966-bad7f24318b2

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Classes are balanced in the training data.

TYPE:         Missing values imputation
STATUS:       FIXED
DESCRIPTION:  The training data had the following missing values which were resolved.

Please review your data source for data quality issues and possibly filter out the rows 

In [45]:
# Widget で結果確認
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …

In [46]:
# 詳細ログの出力
#remote_run.get_details()

In [47]:
#remote_run.get_guardrails()

In [48]:
best_run, fitted_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-classification-hmeq-dev,AutoML_bbacf504-e49c-47d3-9966-bad7f24318b2_0,,Completed,Link to Azure Machine Learning studio,Link to Documentation


### 機械学習モデルの理解

In [49]:
pd.DataFrame(fitted_model.named_steps['datatransformer'].get_engineered_feature_names())

,0
0,融資依頼金額_MeanImputer
1,未払担保金額_MeanImputer
2,未払担保金額_ImputationMarker
3,現在資産価値_MeanImputer
4,現在資産価値_ImputationMarker
5,勤務年数_MeanImputer
6,勤務年数_ImputationMarker
7,信用調査会社問い合わせ数_MeanImputer
8,信用調査会社問い合わせ数_ImputationMarker
9,延滞トレードライン数_MeanImputer


In [50]:
pd.DataFrame.from_records(fitted_model.named_steps['datatransformer'].get_featurization_summary())

,Dropped,EngineeredFeatureCount,RawFeatureName,Transformations,TypeDetected
0,No,1,融資依頼金額,[MeanImputer],Numeric
1,No,2,未払担保金額,"[MeanImputer, ImputationMarker]",Numeric
2,No,2,現在資産価値,"[MeanImputer, ImputationMarker]",Numeric
3,No,2,勤務年数,"[MeanImputer, ImputationMarker]",Numeric
4,No,2,信用調査会社問い合わせ数,"[MeanImputer, ImputationMarker]",Numeric
5,No,2,延滞トレードライン数,"[MeanImputer, ImputationMarker]",Numeric
6,No,2,最も古いトレードラインの月齢,"[MeanImputer, ImputationMarker]",Numeric
7,No,2,最近のクレジットの問い合わせ数,"[MeanImputer, ImputationMarker]",Numeric
8,No,2,トレード(クレジット)ラインの数,"[MeanImputer, ImputationMarker]",Numeric
9,No,2,債務対所得の割合,"[MeanImputer, ImputationMarker]",Numeric


## 3. モデル解釈
Azure Machine Learning には Automated ML のモデルを解釈する仕組みがあります。詳しくは [モデルを解釈する方法](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-machine-learning-interpretability#how-to-interpret-your-model)をご参照ください。

In [51]:
# Pandas Dataframe に変換
train_df = train_dataset.to_pandas_dataframe()
test_df = test_dataset.to_pandas_dataframe()

In [52]:
from azureml.train.automl.runtime.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations
automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, 
                                                             X=train_df.drop([label], axis=1), 
                                                             X_test=test_df.drop([label], axis=1), 
                                                             y=train_df[label].values, 
                                                             task='classification')

Current status: Setting up data for AutoML explanations
Current status: Setting up the AutoML featurizer
Current status: Setting up the AutoML featurization for explanations
Current status: Setting up the AutoML estimator
Current status: Generating a feature map for raw feature importance
Current status: Finding all classes from the dataset
Current status: Data for AutoML explanations successfully setup


`MimicWrapper` は **Global Surrogate Model** によるグローバルなモデル解釈を実現します。ここでは LightGBM を用いています。

In [53]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(workspace, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

## Engineered Explanation
特徴量エンジニアリングで生成された変数を用いたモデル解釈を行います。

In [54]:
# Compute the engineered explanations
engineered_explanations = explainer.explain(['local', 'global'],get_raw=False,
                                            eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [55]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(engineered_explanations, 
                     automl_explainer_setup_obj.automl_estimator, 
                     datasetX=automl_explainer_setup_obj.X_test_transform)

ExplanationWidget(value={'predictedY': [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0…

## Raw Explanation
特徴量エンジニアリング前の変数を用いたモデル解釈を行います。

In [56]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [57]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(raw_explanations, 
                     automl_explainer_setup_obj.automl_pipeline, 
                     datasetX=automl_explainer_setup_obj.X_test_raw)

ExplanationWidget(value={'predictedY': [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0…